### dotenv 불러오기

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['API_KEY']=''


In [1]:
import pandas as pd
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 데이터 로드
file_path = './emergency_hospital_data_final.csv'
df = pd.read_csv(file_path)

# OpenAI 모델 초기화
model = ChatOpenAI(model='gpt-4')  # 원하는 모델명으로 변경 가능

# 진료과목 및 구역을 기반으로 병원 추천하는 함수
def recommend_emergency_room(df, medical_field_input, user_district):
    # 구역 및 진료과목 필터링
    relevant_hospitals = df[
        (df['구역'] == user_district) &
        df.loc[:, '진료과목1':'진료과목36'].apply(lambda x: x.str.contains(medical_field_input, case=False, na=False)).any(axis=1)
    ]
    
    # 병원 정보 정리
    if not relevant_hospitals.empty:
        recommendations = []
        for _, row in relevant_hospitals.iterrows():
            recommendations.append(f"{row['병원']}, 주소: {row['주소']}, 전화번호: {row['전화번호']}")
        return "\n".join(recommendations)
    else:
        return "해당 진료과목에 맞는 응급실 정보가 없습니다."

# LangChain Prompt Template 정의
prompt_template = PromptTemplate(
    input_variables=["medical_field", "district", "recommendations"],
    template=(
        "사용자가 '{medical_field}' 진료과목으로 '{district}' 구역의 응급실을 찾고 있습니다. "
        "다음은 추천되는 병원 목록입니다:\n\n{recommendations}\n\n"
        "이 정보를 바탕으로 사용자에게 응급실 정보를 알려주세요."
    ),
)

# 사용자 요청에 따라 응급실 정보 생성하는 함수
def generate_emergency_response(medical_field_input, user_district):
    # 진료과목과 위치에 따른 추천 병원 필터링
    recommendations = recommend_emergency_room(df, medical_field_input, user_district)
    
    # LangChain을 통한 응답 생성
    chain = LLMChain(llm=model, prompt=prompt_template)
    response = chain.run({
        "medical_field": medical_field_input,
        "district": user_district,
        "recommendations": recommendations
    })
    return response

# 예시 입력
medical_field_input = "내과"  # 사용자가 입력한 진료과목
user_district = "강서구"       # 사용자가 입력한 지역
response_text = generate_emergency_response(medical_field_input, user_district)

print(response_text)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13192\1322531204.py:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=model, prompt=prompt_template)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13192\1322531204.py:47: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain.run({


안녕하세요, '강서구' 내에 위치한 '내과' 진료과목에 대한 응급실 정보를 찾으시는군요. 다음은 몇 가지 추천 병원들입니다.

1. 이화여자대학교의과, 대학부속서울병원
- 위치: 서울특별시 강서구 공항대로 260, 이화의대부속서울병원 (마곡동)
- 전화번호: 1522-7000

2. 강서케이병원
- 위치: 서울특별시 강서구 가로공원로 187, 나누리강서병원 지1층~7층 (화곡동)
- 전화번호: 02-1833-7530

3. 부민병원
- 위치: 서울특별시 강서구 공항대로 389, 부민병원 (등촌동)
- 전화번호: 1577-7582

4. 미즈메디병원
- 위치: 서울특별시 강서구 강서로 295 (내발산동)
- 전화번호: 02-2007-1252

5. 우리들병원
- 위치: 서울특별시 강서구 하늘길 70 (과, 해동)
- 전화번호: 02-2660-7500

꼭 전화로 병원을 먼저 연락하신 후 방문해주세요. 도움이 되셨기를 바랍니다.


In [2]:
medical_field_input = "이비인후과"  # 사용자가 입력한 진료과목
user_district = "강남구"       # 사용자가 입력한 지역
response_text = generate_emergency_response(medical_field_input, user_district)

print(response_text)

강남구 지역에서 이비인후과 응급실을 찾으시는군요. 다음 두 곳을 추천드립니다. 

첫 번째는 '연세대학교의과 대학 강남세브란스병원'입니다. 주소는 서울특별시 강남구 언주로 211, 강남세브란스병원 (도곡동)이며, 전화번호는 02-2019-3114입니다.

두 번째는 '삼성서울병원'입니다. 주소는 서울특별시 강남구 일원로 81 (일원동, 삼성의료원)이며, 전화번호는 02-3410-2114입니다.

응급 상황에서는 빠르게 의료진에게 연락하시는 것이 중요하니, 필요한 정보가 모두 제공되었는지 다시 한번 확인해보세요.


### streamlit에 적용할 파이썬코드

In [ ]:
import pandas as pd
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

os.environ["API_KEY"] = ""

# 데이터 로드
file_path = './data/emergency_hospital_data_final.csv'
df = pd.read_csv(file_path)

# OpenAI 모델 초기화
model = ChatOpenAI(model='gpt-4')

# 진료과목 및 구역을 기반으로 병원 추천하는 함수
def recommend_emergency_room(df, medical_field_input, user_district):
    relevant_hospitals = df[
        (df['구역'] == user_district) & 
        df.loc[:, '진료과목1':'진료과목36'].apply(
            lambda x: x.str.contains(medical_field_input, case=False, na=False)
        ).any(axis=1)
    ]
    
    if not relevant_hospitals.empty:
        recommendations = []
        for _, row in relevant_hospitals.iterrows():
            recommendations.append(f"{row['병원']}, 주소: {row['주소']}, 전화번호: {row['전화번호']}")
        return "\n".join(recommendations)
    else:
        return "해당 진료과목에 맞는 응급실 정보가 없습니다."

# 증상에 따른 응급처치 및 병원 추천 함수
def get_treatment_and_field(symptom_input):
    # 증상 입력에 맞는 응급처치 정보 추출 (샘플로 '내과' 진료과목을 사용)
    symptom_input_clean = symptom_input.lower().strip()
    matched_row = df[df['증상'].str.contains(symptom_input_clean, case=False, na=False)]
    
    if not matched_row.empty:
        symptom = matched_row.iloc[0]['증상']
        emergency_action = matched_row.iloc[0]['응급처치']
        related_field1 = matched_row.iloc[0]['진료과목1']
        related_field2 = matched_row.iloc[0]['진료과목2']
        
        return symptom, emergency_action, related_field1, related_field2
    else:
        return None, None, None, None

# LangChain Prompt Template 정의
prompt_template = PromptTemplate(
    input_variables=["medical_field", "district", "recommendations"],
    template=(
        "사용자가 '{medical_field}' 진료과목으로 '{district}' 구역의 응급실을 찾고 있습니다. "
        "다음은 추천되는 병원 목록입니다:\n\n{recommendations}\n\n"
        "이 정보를 바탕으로 사용자에게 응급실 정보를 알려주세요."
    ),
)

# 응급실 추천 정보를 생성하는 함수
def generate_emergency_response(medical_field_input, user_district):
    # 추천 병원 정보 가져오기
    recommendations = recommend_emergency_room(df, medical_field_input, user_district)
    
    # LangChain을 통한 응답 생성
    chain = LLMChain(llm=model, prompt=prompt_template)
    response = chain.run({
        "medical_field": medical_field_input,
        "district": user_district,
        "recommendations": recommendations
    })
    return response

# 테스트 호출 (예시 입력)
if __name__ == "__main__":
    medical_field_input = "내과"  # 진료과목
    user_district = "강서구"       # 지역
    response_text = generate_emergency_response(medical_field_input, user_district)
    print(response_text)
